***Méthodes numériques: Module 4***

# Grey Scott model

Nous commencons par importer la librairie numpy, matplotlib et matplotlib.cm qui nous permettra de dessiner des **cartes**.
Ensuite nous définissons tous nos paramètres et nous importons le fichier contenant les données initiales u et v qui se trouve dans le fichier data du même dépôt.

In [1]:
import numpy
from matplotlib import pyplot
import matplotlib.cm as cm
%matplotlib inline

In [2]:
n = 192
Du, Dv, F, k = 0.00016, 0.00008, 0.035, 0.065 # Bacteria 1 
dh = 5/(n-1)
T = 8000
dt = .9 * dh**2 / (4*max(Du,Dv))
nt = int(T/dt)

In [3]:
uvinitial = numpy.load('./data/uvinitial.npz')
U = uvinitial['U']
V = uvinitial['V']

Le système d'équations différentielles à résoudre s'écrit:


\begin{align}
\frac{\partial u}{\partial t} &= D_u \nabla ^2 u - uv^2 + F(1-u)\\
\frac{\partial v}{\partial t} &= D_v \nabla ^2 v + uv^2 - (F + k)v
\end{align}

## Méthode explicite

Nous allons le discrétiser vers l'avant dans le temps et par un schéma centré dans l'espace (forward-time,centred-space). Nous allons le résoudre par un schéma explicite: la discrétisation dans l'espace s'effectue au temps n:


\begin{align}
\frac{u_{i,j}^{n+1}-u_{i,j}^{n}}{\Delta t}= D_{u}*(\frac{u_{i+1,j}^{n}-2u_{i,j}^{n}+u_{i-1,j}^{n}}{\Delta x^2}+\frac{u_{i,j+1}^{n}-2u_{i,j}^{n}+u_{i,j-1}^{n}}{\Delta y^2})-u_{i,j}^{n}*(v_{i,j}^{n})^2+F(1-u_{i,j}^{n})\\
\frac{v_{i,j}^{n+1}-v_{i,j}^{n}}{\Delta t}=D_{v}*(\frac{v_{i+1,j}^{n}-2v_{i,j}^{n}+v_{i-1,j}^{n}}{\Delta x^2}+\frac{v_{i,j+1}^{n}-2v_{i,j}^{n}+v_{i,j-1}^{n}}{\Delta y^2})+u_{i,j}^{n}*(v_{i,j}^{n})^2-F(1+k)v_{i,j}^{n}
\end{align}

On pose ensuite 
\begin{align}
\Delta x^2=\Delta y^2=\delta^2\\
\Delta t= \frac{9}{40}\frac{\delta^2}{\max(D_u, D_v)}
\end{align}

On met tous les termes connus dans le membre de droite et les termes inconnus dans le membre de gauche:

\begin{align}
u_{i,j}^{n+1}=u_{i,j}^{n}+\Delta t(\frac{u_{i+1,j}^{n}-2u_{i,j}^{n}+u_{i-1,j}^{n}}{\delta^2}+\frac{u_{i,j+1}^{n}-2u_{i,j}^{n}+u_{i,j-1}^{n}}{\delta^2}-u_{i,j}^{n}*(v_{i,j}^{n})^2+F(1-u_{i,j}^{n}))\\
v_{i,j}^{n+1}=v_{i,j}^{n}+\Delta t(\frac{v_{i+1,j}^{n}-2v_{i,j}^{n}+v_{i-1,j}^{n}}{\delta^2}+\frac{v_{i,j+1}^{n}-2v_{i,j}^{n}+v_{i,j-1}^{n}}{\delta^2}+u_{i,j}^{n}*(v_{i,j}^{n})^2-F(1+k)v_{i,j}^{n})
\end{align}

### Paramètres:

In [4]:
n = 192
Du, Dv, F, k = 0.00016, 0.00008, 0.035, 0.065 # Bacteria 1 
dh = 5/(n-1) #delta
T = 8000
dt = .9 * dh**2 / (4*max(Du,Dv))
nt = int(T/dt)

In [7]:
def greyscott(u0,v0):
    ucopy=u0.copy()
    vcopy=v0.copy()
    for k in range (3):
        uk=ucopy.copy()
        vk=vcopy.copy()
        ucopy[1:-1,1:-1]=uk[1:-1,1:-1]*(1-4*dt/(dh**2))+dt*((uk[2:,1:-1]+uk[:-2,1:-1]+uk[1:-1,2:]+uk[1:-1,:-2])/(dh**2)\
                        -uk[1:-1,1:-1]*(vk[1:-1,1:-1])**2+F*(1-uk[1:-1,1:-1]))
        vcopy[1:-1,1:-1]=vk[1:-1,1:-1]*(1-4*dt/(dh**2))+dt*((vk[2:,1:-1]+vk[:-2,1:-1]+vk[1:-1,2:]+vk[1:-1,:-2])/(dh**2)\
                        +uk[1:-1,1:-1]*(vk[1:-1,1:-1])**2-F*(1+k)*vk[1:-1,1:-1])
        #conditions de Neumann:
        ucopy[:,0]=ucopy[:,1]
        ucopy[0,:]=ucopy[1,:]
        ucopy[:,191]=ucopy[:,190]
        ucopy[191,:]=ucopy[190,:]
        vcopy[:,0]=vcopy[:,1]
        vcopy[0,:]=vcopy[1,:]
        vcopy[:,191]=vcopy[:,190]
        vcopy[191,:]=vcopy[190,:]
    return (ucopy,vcopy)

In [8]:
sol=greyscott(U,V)
print(sol[0])

[[ -3.41262453e+17  -3.41262453e+17  -9.59111018e+15 ...,   1.25856735e+15
   -1.40699879e+15  -1.40699879e+15]
 [ -3.41262453e+17  -3.41262453e+17  -9.59111018e+15 ...,   1.25856735e+15
   -1.40699879e+15  -1.40699879e+15]
 [ -1.04030203e+17  -1.04030203e+17   4.74053011e+17 ...,  -5.19765668e+16
    2.30314610e+15   2.30314610e+15]
 ..., 
 [  1.04445401e+17   1.04445401e+17   4.55448172e+17 ...,   1.77162735e+16
   -1.02443211e+18  -1.02443211e+18]
 [ -4.88865044e+19  -4.88865044e+19  -6.88000720e+16 ...,   9.16397612e+16
   -3.14756464e+18  -3.14756464e+18]
 [ -4.88865044e+19  -4.88865044e+19  -6.88000720e+16 ...,   9.16397612e+16
   -3.14756464e+18  -3.14756464e+18]]


## Méthode implicite

Nous allons le résoudre par un schéma implicite: la discrétisation dans l'espace s'effectue au temps n+1:

$$
\begin{align}
\frac{u_{i,j}^{n+1}-u_{i,j}^{n}}{\Delta t}= D_{u}*(\frac{u_{i+1,j}^{n+1}-2u_{i,j}^{n+1}+u_{i-1,j}^{n+1}}{\Delta x^2}+\frac{u_{i,j+1}^{n+1}-2u_{i,j}^{n+1}+u_{i,j-1}^{n+1}}{\Delta y^2})-u_{i,j}^{n}*(v_{i,j}^{n})^2+F(1-u_{i,j}^{n})\\
\frac{v_{i,j}^{n+1}-v_{i,j}^{n}}{\Delta t}=D_{v}*(\frac{v_{i+1,j}^{n+1}-2v_{i,j}^{n+1}+v_{i-1,j}^{n+1}}{\Delta x^2}+\frac{v_{i,j+1}^{n+1}-2v_{i,j}^{n+1}+v_{i,j-1}^{n+1}}{\Delta y^2})+u_{i,j}^{n}*(v_{i,j}^{n})^2-F(1+k)v_{i,j}^{n})
\end{align}
$$
Le système d'équations différentielles à résoudre s'écrit:
$$
\begin{align}
\frac{\partial u}{\partial t} &= D_u \nabla ^2 u - uv^2 + F(1-u)\\
\frac{\partial v}{\partial t} &= D_v \nabla ^2 v + uv^2 - (F + k)v
\end{align}
$$
Nous allons le discrétiser vers l'avant dans le temps et par un schéma centré dans l'espace (forward-time,centred-space). Nous commençons par le résoudre à l'aide d'un schéma explicite: la discrétisation dans l'espace s'effectue au temps n:
$$
\begin{align}
\frac{u_{i,j}^{n+1}-u_{i,j}^{n}}{\Delta t}= D_{u}*(\frac{u_{i+1,j}^{n}-2u_{i,j}^{n}+u_{i-1,j}^{n}}{\Delta x^2}+\frac{u_{i,j+1}^{n}-2u_{i,j}^{n}+u_{i,j-1}^{n}}{\Delta y^2})-u_{i,j}^{n}*(v_{i,j}^{n})^2+F(1-u_{i,j}^{n})\\
\frac{v_{i,j}^{n+1}-v_{i,j}^{n}}{\Delta t}=D_{v}*(\frac{v_{i+1,j}^{n}-2v_{i,j}^{n}+v_{i-1,j}^{n}}{\Delta x^2}+\frac{v_{i,j+1}^{n}-2v_{i,j}^{n}+v_{i,j-1}^{n}}{\Delta y^2})+u_{i,j}^{n}*(v_{i,j}^{n})^2-F(1+k)v_{i,j}^{n})
\end{align}
$$